In [5]:
B = [[2, 5, 6], [np.nan, 0, 7], [9, np.nan, 7]]
A = np.random.random((3, 3))

In [6]:
import numpy as np
import numpy.ma as ma

b=ma.masked_invalid(B)

msk = ~b.mask

print(ma.corrcoef(A[msk],b[msk]))

[[1.0 -0.6042021829542006]
 [-0.6042021829542006 1.0]]


In [7]:
msk

array([[ True,  True,  True],
       [False,  True,  True],
       [ True, False,  True]])

In [8]:
A = np.random.random((744, 81, 161, 5))